In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [11]:
root = "../data/processed/birdclef-2023/train_embeddings"
! ls {root}
! ls {root}/embeddings/*/*/*.parquet | wc -l

audio  consolidated_v1	consolidated_v2_sample	embeddings
5498


In [2]:
from birdclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark(cores=16, memory="12g")
df = spark.read.parquet(f"{root}/embeddings/*/*/*.parquet")
df.printSchema()
df.count()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 03:02:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/23 03:02:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# write this out to a new location
df.repartition(32).write.parquet(f"{root}/consolidated_v2", mode="overwrite")

In [8]:
df = spark.read.parquet(f"{root}/consolidated_v2")

In [9]:
df.select("track_name").distinct().count()

845

In [10]:
# upload the dataset to gcs
! bash -c "gsutil -m rsync -r {root}/consolidated_v2 gs://birdclef-2023/data/processed/birdclef-2023/train_embeddings_consolidated_v2"

Building synchronization state...
Starting synchronization...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated_v2_sample/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated_v2_sample/.part-00002-7f37ce3d-1941-4938-b673-1e0463a3a05b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated_v2_sample/.part-00000-7f37ce3d-1941-4938-b673-1e0463a3a05b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated_v2_sample/.part-00001-7f37ce3d-1941-4938-b673-1e0463a3a05b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://../data/processed/birdclef-2023/train_embeddings/consolidated_v2_sample/.part-00010-7f37ce3d-1941-4938-b673-1e0463a3a05b-c000.snappy.parquet.crc [Content-Type=application